Carré de Polybe
===============

Réouvrir la page principale
---------------------------

[Cliquer ici](../main.ipynb)


À vous de jouer : coder un texte "à la Polybe"
----------------------------------------------

Nous redonnons le tableau pour coder nos textes.

         1 | 2   | 3 | 4 | 5
    ------------------------
    1 || a | b   | c | d | e
    2 || f | g   | h | i | j
    3 || k | l   | m | n | o 
    4 || p | q   | r | s | t 
    5 || u | v,w | x | y | z 


Le programme suivant est une traduction directe de la technique de codage proposé par Polybe. Rappelons que les espaces devaient être codés par $00$. Notez l'utilisation de la "pythonnerie" `colonne, ligne = a, b` qui équivaut à l'emploi de `colonne = a` et `ligne = b`. Pour vérifier notre programme, notons au passage que "les" est codé par `235144`.

In [ ]:
# ------------ #
# -- CODAGE -- #
# ------------ #

ALPHABET = "abcdefghijklmnopqrstuvwxyz"


CODAGE = {" ": "00"}

colonne, ligne = 0, 1

for cara in ALPHABET:
    if cara != "w":    
        colonne += 1

        if colonne == 6:
            colonne, ligne = 1, ligne + 1

    CODAGE[cara] = "{0}{1}".format(colonne, ligne)


def code(texte):
    """
Cette fonction crypte un texte via le code de Polybe.
    """
    global CODAGE
    
    texte_code = ""

    for cara in texte:
        texte_code += CODAGE[cara]
    
    return texte_code


# ----------------- #
# -- APPLICATION -- #
# ----------------- #

texte      = "les velibs de polybe"
texte_code = code(texte)

print(texte_code)

Nous retrouvons bien `235144` au début. Dans la section suivante, nous vérifions plus sérieusement la validité de ce code.


Pour les plus rapides : décoder un texte "à la Polybe"
------------------------------------------------------

Rappelons que l'on sait que le codeur utilise $00$ pour les espaces, et que le texte initial ne contenait que des lettres minuscules non accentuées et des espaces, et donc aucun caractère spécial du type point, apostrophe, guillemet ouvrant... etc. Commençons par une première proposition incomplète car elle ne gère pas le cas du "v" et du "w" : on choisit pour le moment de ne mettre que des "v".

In [2]:
# -------------- #
# -- DÉCODAGE -- #
# -------------- #

ALPHABET = "abcdefghijklmnopqrstuvwxyz"


CODAGE = {" ": "00"}

colonne, ligne = 0, 1

for cara in ALPHABET:
    if cara != "w":    
        colonne += 1

        if colonne == 6:
            colonne, ligne = 1, ligne + 1

    CODAGE[cara] = "{0}{1}".format(colonne, ligne)


DECODAGE = {}

for cara, un_code in CODAGE.items():
    # On ne garde pas le "w".
    if un_code not in DECODAGE:
        DECODAGE[un_code] = cara

        
def decode(texte_code):
    """
Cette fonction décode partiellement un texte codé via le code
de Polybe. Le cas ambigu des "v" et des "w" n'est pas traité
ici, plus précisément les mots ne contiendront aucun "w".
    """
    global DECODAGE
    
    taille_texte = len(texte_code)
    texte_decode = ""

    for pos in range(0, taille_texte, 2):
        un_code       = texte_code[pos:pos+2]
        texte_decode += DECODAGE[un_code]

    return texte_decode


# ----------------- #
# -- APPLICATION -- #
# ----------------- #

texte_code   = "5451355451002142514300415131534151002134112553"
texte_decode = decode(texte_code)

print('Texte codé   : "{0}"'.format(texte_code))
print('Texte décodé : "{0}"'.format(texte_decode))

Texte codé   : "5451355451002142514300415131534151002134112553"
Texte décodé : "texte bien decode bravo"


Ceci semble être juste. Vérifions maintenant que le code trouvé dans la première section était bien le bon *(au passage, nous testons en fait nos deux programmes)*.

In [3]:
texte_code   = "2351440025512342214400415100145323452151"
texte_decode = decode(texte_code)

print('Texte initial : "les velibs de polybe"')
print('Texte décodé  : "{0}"'.format(texte_decode))

Texte initial : "les velibs de polybe"
Texte décodé  : "les velibs de polybe"


Rassurant mais nous avons eu de la chance jusqu'ici car dans les textes initiaux n'appraissaient pas de "w". Il nous reste donc à gérer le cas d'un texte où appraissent des "v" et des "w". Le code suivant montre la faiblesse de la première version de notre programme décodeur. Indiquons au passage [cette page](http://www.listesdemots.fr/accueil.htm) qui permet d'obtenir des mots vérifiant certaines propriétés. 

In [4]:
# On suppose que touts les cellules précédentes ont été exécutées
# par IPython. Ceci permet d'utiliser les fonctions code et decode.

texte        = "interview du wagon et de la voiture"
texte_code   = code(texte)
texte_decode = decode(texte_code)

print("Texte initial       :", texte)
print('Texte "codé-décodé" :', texte_decode)

Texte initial       : interview du wagon et de la voiture
Texte "codé-décodé" : interviev du vagon et de la voiture


Rappelons la concrainte forte qui dit que les espaces sont codés par $00$. Ceci nous permet d'isoler facilement les mots dans notre texte décodé imparfaitement. Nous allons donc construire une fonction qui pour chaque mot contenant des "v" tentera de retrouver un mot existant obtenu en remplaçant certains "v" par des "w".  À vous de voir comment fonctionne le code ci-après *(notez au passage l'utilisation des méthodes `split` et `join` des variables de type `str`, ainsi que l'emploi de `raise` pour envoyer un message d'erreur en cas d'échec)*.

**Remarque : ** nous restons dans le cas de mots sans accent mais étendre le code ci-après pour gérer plus de caractères n'est pas une tâche ardue.

In [5]:
# On suppose que touts les cellules précédentes ont été exécutées
# par IPython. Ceci permet d'utiliser les fonctions code et decode.

# -------------------------------------------------- #
# -- CHOISIR ENTRE "V" ET "W" (cf. "Palindromes") -- #
# -------------------------------------------------- #

MOTS_TOUT_EN_V = {}

with open("motsfrancais_frgut.txt", encoding="iso-8859-1") as fichier:
    for ligne in fichier:
        mot = ligne.strip()
        mot = mot.lower()
        
        if "v" in mot or "w" in mot:
            mot_tout_en_v = mot.replace("w", "v")
            
            if mot_tout_en_v in MOTS_TOUT_EN_V:
                MOTS_TOUT_EN_V[mot_tout_en_v].append(mot)
            else:
                MOTS_TOUT_EN_V[mot_tout_en_v] = [mot]

        
def bonchoix_vw(texte):
    """
Cette fonction tente de reconstruire un texte où tous les "w" 
ont été remplacés par des "v".
    """
    global MOTS_TOUT_EN_V
    
    # On "éclate" le texte suivant chaque espace.
    mots = texte.split(" ")
    
    mots_trouves = []

    for un_mot in mots:
        if un_mot in MOTS_TOUT_EN_V:
            mots_connus_associes = MOTS_TOUT_EN_V[un_mot]

            if len(mots_connus_associes) > 1:
                raise Exception("Décodage impossible !")
                
            un_mot = mots_connus_associes[0]
            
        mots_trouves.append(un_mot)
        
    # On retourne le texte obtenu en "collant", avec un espace comme
    # jointure, les mots contenus dans la variable mots_trouves. 
    return " ".join(mots_trouves)
            
            
# ----------------- #
# -- APPLICATION -- #
# ----------------- #

texte      = "interview du wagon et de la voiture"
texte_code = code(texte)

texte_decode_partiellement = decode(texte_code)
texte_decode_totalement    = bonchoix_vw(texte_decode_partiellement)


print("Texte initial              :", texte)
print('Texte décodé partiellement :', texte_decode_partiellement)
print('Texte décodé totalement    :', texte_decode_totalement)

Texte initial              : interview du wagon et de la voiture
Texte décodé partiellement : interviev du vagon et de la voiture
Texte décodé totalement    : interview du wagon et de la voiture
